In [6]:
import os
import glob

In [28]:
data_path = './data'
train_data_path = os.path.join(data_path, 'Train')
java_folder_path = train_data_path + '/*.java'
        
file_pair_list = []

for _, folder_name in enumerate(glob.glob(train_data_path + '/*/')):
    file_pair = []
    for _, file_name in enumerate(glob.glob(folder_name + '/*.java')):
        file_pair.append(file_name)
    
    file_pair_list.append(file_pair)

with open("C:\\Users\\emili\\Documents\\Tec\\Laboratorio\\TC3002B\\repos\\TC3002B_Reto_DevDiv\\data\\Train\\00af3420_5449d33c\\00af3420.java", 'r') as file1:
    file1_string = file1.read().replace('\n', '')
    print(file1_string)

[['./data\\Train\\0017d438_9852706b\\0017d438.java', './data\\Train\\0017d438_9852706b\\9852706b.java'], ['./data\\Train\\0017d438_ac180326\\0017d438.java', './data\\Train\\0017d438_ac180326\\ac180326.java'], ['./data\\Train\\003.java-004.java\\003.java', './data\\Train\\003.java-004.java\\004.java'], ['./data\\Train\\0048a372_0adb1ee5\\0048a372.java', './data\\Train\\0048a372_0adb1ee5\\0adb1ee5.java'], ['./data\\Train\\005.java-006.java\\005.java', './data\\Train\\005.java-006.java\\006.java'], ['./data\\Train\\008.java-010.java\\008.java', './data\\Train\\008.java-010.java\\010.java'], ['./data\\Train\\00af3420_5449d33c\\00af3420.java', './data\\Train\\00af3420_5449d33c\\5449d33c.java'], ['./data\\Train\\00af3420_86102d81\\00af3420.java', './data\\Train\\00af3420_86102d81\\86102d81.java'], ['./data\\Train\\00af3420_d92c5342\\00af3420.java', './data\\Train\\00af3420_d92c5342\\d92c5342.java'], ['./data\\Train\\00af3420_f4d6d28d\\00af3420.java', './data\\Train\\00af3420_f4d6d28d\\f4d6d2